In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
## functions necessary to group the minute level to hours/days
def get_first_element(x):
    return x.iloc[0]

def get_last_element(x):
    return x.iloc[len(x)-1]

# indicate the indicator as positive when the price is within specified threshold 
##ranges compared to the open for the long strategy
def infer_profit_long_indicator(x,threshold=1.02):
    if max(x['next_1high'],x['High']) >= threshold*x['Open']:
    #if x['Close'] >= threshold*x['Open']:
        return 1
    else:
        return 0
def infer_profit_short_indicator(x,threshold=0.98):
    if min(x['next_1low'],x['Low']) <= threshold*x['Open']:
    #if x['Close'] <= threshold*x['Open']:
        return 1
    else:
        return 0

def compute_long_spread(x):
    return round((max(x['next_1high'],x['High'])-x['Open'])/x['Open'],4)*100
def compute_short_spread(x):
    return round((x['Open']-min(x['next_1low'],x['Low']))/x['Open'],4)*100
    
def zscore_func_improved(x,window_size=20):
    rolling_mean=x.rolling(window=window_size).mean().bfill()
    rolling_std = x.rolling(window=window_size).std().bfill()
    return (x-rolling_mean)

## Preparing FCPO Data

### Process Back Adjusted Data

In [4]:
#pre-process the data from 2014-2018 csv file
fcpo_data_badjusted=pd.read_csv('data/FCPO_2007-2017_backadjusted.csv')

#remove identity placeholder columns
fcpo_data_badjusted=fcpo_data_badjusted[fcpo_data_badjusted['Time']!=1805]

#convert the date time values to string and derive the hour field
fcpo_data_badjusted['Date']=fcpo_data_badjusted['Date'].apply(lambda x:str(x))
fcpo_data_badjusted['Hour']=fcpo_data_badjusted['Time'].apply(lambda x:str(x)[0:2])

fcpo_daily_badjusted=fcpo_data_badjusted.groupby(['Date'],as_index=False).agg({'Open':lambda x: get_first_element(x),
                              'High':lambda x:np.max(x),'Low':lambda x:np.min(x),'Close': lambda x:get_last_element(x),'Volume':'sum'})

fcpo_daily_badjusted=fcpo_daily_badjusted.set_index(pd.to_datetime(fcpo_daily_badjusted['Date']))
fcpo_daily_badjusted=fcpo_daily_badjusted.drop(columns=['Date'])

# Compute hourly data frames
fcpo_hourly_badjusted=fcpo_data_badjusted.groupby(['Date','Hour'],as_index=False).agg({'Open':lambda x: get_first_element(x),'Close': lambda x:get_last_element(x),
                              'High':lambda x:np.max(x),'Low':lambda x:np.min(x),'Volume':'sum'})

fcpo_hourly_badjusted=fcpo_hourly_badjusted.set_index(pd.to_datetime(fcpo_hourly_badjusted['Date']+'-'+fcpo_hourly_badjusted['Hour']))
fcpo_hourly_badjusted=fcpo_hourly_badjusted.drop(columns=['Date','Hour'])

In [5]:
# shift the output of next day into the daily data frame
fcpo_daily_badjusted=fcpo_daily_badjusted.assign(
                      next_1open=fcpo_daily_badjusted['Open'].shift(-1),
                      next_1high=fcpo_daily_badjusted['High'].shift(-1),
                      next_1low=fcpo_daily_badjusted['Low'].shift(-1),                      
                      next_1close=fcpo_daily_badjusted['Close'].shift(-1),
                      prev_1open=fcpo_daily_badjusted['Open'].shift(1), 
                      prev_1high=fcpo_daily_badjusted['High'].shift(1), 
                      prev_1low=fcpo_daily_badjusted['Low'].shift(1), 
                      prev_1close=fcpo_daily_badjusted['Close'].shift(1),                 
                      )
## infer the profit indicators based on the future closing/high or low  prices
# compute the long-profit and short-profit indicators
# compute for each day, the open change pct compared to the previoius day
# compute for each day, the next day open change pct, compared to current day close (only use for evaluation/validation purposes)
fcpo_daily_badjusted=fcpo_daily_badjusted.assign(
                            prev_open_change_pct=(fcpo_daily_badjusted['Open']/fcpo_daily_badjusted['prev_1close'])*100-100,
                            next_open_change_pct=(fcpo_daily_badjusted['next_1open']/fcpo_daily_badjusted['Close'])*100-100,                   
                            lprofit_ind_1pct=fcpo_daily_badjusted.apply(lambda x:infer_profit_long_indicator(x,1.01),axis=1),
                            sprofit_ind_1pct=fcpo_daily_badjusted.apply(lambda x:infer_profit_short_indicator(x,0.99),axis=1),
                            lprofit_ind_2pct=fcpo_daily_badjusted.apply(lambda x:infer_profit_long_indicator(x,1.02),axis=1),
                            sprofit_ind_2pct=fcpo_daily_badjusted.apply(lambda x:infer_profit_short_indicator(x,0.98),axis=1)
                            )

fcpo_daily_badjusted['prev_open_change_pct']=fcpo_daily_badjusted['prev_open_change_pct'].bfill()

In [6]:
#fcpo_data_hourly=fcpo_data_2014_2018_hourly.copy()
## infer the profit indicators based on the future closing prices
# shift the output of next 3 time periods into the daily data frame
fcpo_hourly_badjusted=fcpo_hourly_badjusted.assign(
                      next_1high=fcpo_hourly_badjusted['High'].shift(-1),
                      next_1low=fcpo_hourly_badjusted['Low'].shift(-1),
                      next_1open=fcpo_hourly_badjusted['Open'].shift(-1),
                     next_1close=fcpo_hourly_badjusted['Close'].shift(-1),)

fcpo_hourly_badjusted=fcpo_hourly_badjusted.assign(lprofit_ind=fcpo_hourly_badjusted.apply(lambda x:infer_profit_long_indicator(x,1.01),axis=1),
                                       sprofit_ind=fcpo_hourly_badjusted.apply(lambda x:infer_profit_short_indicator(x,0.99),axis=1))

### Process Non-Backadjusted Data

In [7]:
fcpo_daily_nadjusted=pd.read_csv('data/fcpo_daily_2018.csv')
fcpo_daily_nadjusted=fcpo_daily_nadjusted[['Date','Open','High','Low','Close','Volume']]
fcpo_daily_nadjusted=fcpo_daily_nadjusted.set_index(pd.to_datetime(fcpo_daily_nadjusted['Date']))
fcpo_daily_nadjusted=fcpo_daily_nadjusted.drop(columns=['Date'])

In [25]:
fcpo_daily_nadjusted=fcpo_daily_nadjusted.assign(
                      next_1open=fcpo_daily_nadjusted['Open'].shift(-1),
                      next_1high=fcpo_daily_nadjusted['High'].shift(-1),
                      next_1low=fcpo_daily_nadjusted['Low'].shift(-1),                      
                      next_1close=fcpo_daily_nadjusted['Close'].shift(-1),
                      prev_1open=fcpo_daily_nadjusted['Open'].shift(1), 
                      prev_1high=fcpo_daily_nadjusted['High'].shift(1), 
                      prev_1low=fcpo_daily_nadjusted['Low'].shift(1), 
                      prev_1close=fcpo_daily_nadjusted['Close'].shift(1),                 
                      )
## infer the profit indicators based on the future closing/high or low  prices
# compute the long-profit and short-profit indicators
# compute for each day, the open change pct compared to the previoius day
# compute for each day, the next day open change pct, compared to current day close (only use for evaluation/validation purposes)
fcpo_daily_nadjusted=fcpo_daily_nadjusted.assign(
                            prev_open_change_pct=(fcpo_daily_nadjusted['Open']/fcpo_daily_nadjusted['prev_1close'])*100-100,
                            next_open_change_pct=(fcpo_daily_nadjusted['next_1open']/fcpo_daily_nadjusted['Close'])*100-100,                   
                            lprofit_ind_2pct=fcpo_daily_nadjusted.apply(lambda x:infer_profit_long_indicator(x,1.02),axis=1),
                            sprofit_ind_2pct=fcpo_daily_nadjusted.apply(lambda x:infer_profit_short_indicator(x,0.98),axis=1),
                            lprofit_ind=fcpo_daily_nadjusted.apply(lambda x:infer_profit_long_indicator(x,1.0125),axis=1),
                            sprofit_ind=fcpo_daily_nadjusted.apply(lambda x:infer_profit_short_indicator(x,0.9875),axis=1),    
                            long_spread = fcpo_daily_nadjusted.apply(lambda x: compute_long_spread(x),axis=1),
                            short_spread = fcpo_daily_nadjusted.apply(lambda x: compute_short_spread(x),axis=1),
                            )
fcpo_daily_nadjusted['prev_open_change_pct']=fcpo_daily_nadjusted['prev_open_change_pct'].bfill()

## Preparing Dalian Palm oil data

In [9]:
palmoil_data=pd.read_excel('data/Dalian Palm Olein 1 Mins_updated.xlsx',skiprows=3)
palmoil_data=palmoil_data.drop(0)

palmoil_data['Date']=palmoil_data['Dates'].dt.date
palmoil_data['Hour']=palmoil_data['Dates'].dt.hour

# Compute daily and hourly data frames
palmoil_daily=palmoil_data.groupby(['Date'],as_index=False).agg({'Open':lambda x: get_first_element(x),'Close': lambda x:get_last_element(x),
                              'High':lambda x:np.max(x),'Low':lambda x:np.min(x),'Volume':'sum'})
palmoil_hourly=palmoil_data.groupby(['Date','Hour'],as_index=False).agg({'Open':lambda x: get_first_element(x),'Close': lambda x:get_last_element(x),
                              'High':lambda x:np.max(x),'Low':lambda x:np.min(x),'Volume':'sum'})

palmoil_daily=palmoil_daily.set_index(pd.to_datetime(palmoil_daily['Date']))

### Store the Processed data in HDF5 format for pandas

In [27]:
hdf_store=pd.HDFStore('data/processed_dta.h5')

hdf_store.put('fcpo_daily_badjusted',fcpo_daily_badjusted)
hdf_store.put('fcpo_hourly_badjusted',fcpo_hourly_badjusted)
hdf_store.put('fcpo_daily_nadjusted',fcpo_daily_nadjusted)
hdf_store.put('palmoil_daily',palmoil_daily)

hdf_store.close()